# Theoretical background
## Optimal transport
Consider two histograms $a\in \Sigma_m\equiv\{ a\in\mathbb{R}^m_+ : \sum_i a_i=1\}$, and $b\in \Sigma_n\equiv \{ b\in\mathbb{R}^n_+ : \sum_i b_i=1\}$. In our application they will correspond to the distribution of brightness of respectively $n, m$ objects observed in two distinct images.

Moreover suppose we have some cost associated with transporting a unit of mass between any two objects corresponding to the entries of the two histograms. This is e.g. the Euclidean distance between the centres of the two objects.
Store these in a matrix $C\in\mathbb{R}_+^{m\times n}$ with $C_{i,j}$ being the cost of transporting from $i$-th point in the first histogram, to $j$-th in the second one.

Then we consider the problem of 'transporting' one histogram onto the other that minimises the transport cost. We can encode a transport plan in a matrix $P\in\mathbb{R}_+^{m\times n}$ such that the sum of each row and column equals the appropriate index of one of the two histograms, i.e. $P\in U(a,b)\equiv\{P\in\mathbb{R}_+^{m\times n}P\mathbb{1}_n=a, P^T\mathbb{1}_m=b \}$.

Then the problem we are looking to solve is finding $L_C(a,b)=\min\langle C, P\rangle \equiv \sum_{i,j}C_{i,j}P_{i,j}$ over $P\in U(a,b)$


## Unbalanced transport
In many practical scenarios mass conservation may be violated. We may want to allow for that in our framework as follows. Given two histograms $a,b$, we will be considering transport between arbitrary histograms $\tilde{a},\tilde{b}$ in $\Sigma_m,\Sigma_n$ and additionally penalise the difference betwee those and the initial histograms using some divergences $D(a,\tilde{a}) and D(b,\tilde{b})$.

Then we are looking to find
$$
L_C^\tau (a,b)=\min_{\tilde{a},\tilde{b}} L_c(\tilde{a},\tilde{b})+\tau_1 D(a,\tilde{a})+\tau_2 D(b,\tilde{b})= \min_{P\in\mathbb{R}_+^{m\times n}} \langle C,P \rangle+\tau_1 D(P\mathbb{1}_n , a)+\tau_2 D(P^T\mathbb{1}_m , b)
$$

## Efficient computation

With an appropriate choice of a divergence, namely $D=KL$, the Kulback-Leibler divergence and an addition of a regularization term $\epsilon H(P)$ to the minimised sum, we can solve the problem very efficiently. Here $H(P)\equiv -\sum_{i,j}P_{i,j}(log(P_{i,j})-1)$ is the discrete entropy of the coupling matrix.
In this case one can show that the problem becomes convex, the optimal copling matrix is of the form $P_{i,j}=u_i,K_{i,j}v_j$ where $K_{i,j}=e^{-\frac{C_{i,j}}{\epsilon}}$ and the two vectors $u,v$ can be found with so called Sinkhorn iterations:

$$
u \leftarrow \left(\frac{a}{Kv} \right)^{\frac{\tau_1}{\tau_1+\epsilon}} 
$$
$$
v \leftarrow \left(\frac{b}{K^T u} \right)^{\frac{\tau_2}{\tau_2+\epsilon}} 
$$
 
For more details consult e.g. [Gabriel Peyré, Marco Cuturi](https://arxiv.org/abs/1803.00567).

# Algorithm

In a nutshell the way the program works as follows : 

Suppose we have a list of images of some system of point objects which we want to track.

1. In an external program, locate the objects in every frame and return them in a dataframe that contains the frame number, position and brightness/some other mass-equivalent quantity of each object
2. Accounting for the possibiity of an object being missed in some frames, say less than $\Delta N$ of them at a time, compute a cost matrix C for every pair of frames whose number differs by no more then $\Delta N$. The cost can be e.g. the square Euclidean distance between the objects (assuming the frame of reference is stationary)
3. For any cost matrix C, solve the unbalanced OT problem and find the corresponding P
4. Use these matrices to assign a weight to creating a 'connection' between observations in two frames i.e. identifying a pair of objects seen in them. Intuitively, the larger the entry $C_{i,j}$ of a given matrix, the more likely it is that we wish to connect the objects it corresponds to.
5. Sort the possible connections by decreasing weight.
6. While there are connections left to be considered, look at the lowest non-considered connection. If it is s.t. creating it would not interfere with any previously created tracks, then connect the two objects, otherwise reject it and move on.

At the end of that procedue we are left with a dataframe that for each observation contains its track id

# Most important parameters

1. $\Delta N$ referred to in the code as *dt_max* is the maximum separation between frames we consider connecting
2. $\epsilon$ strength of the entropic regularization. For best results it should be as small as possible while preserving quick convergence. In a dataset where the entries of C_{i,j} are on the order of 100, setting it to rougly 0.1 seems to yield good results.\
TODO : How should this scale according to theory?
3. $\tau$ referred to in the code as *alpha* is the weight assigned to the divergence terms in unbalanced OT. We only consider the case of $\tau_1=\tau_2=\tau$.\
Limiting cases : \
 $\tau\rightarrow 0$ : modifying the measures is very cheap so the solution will be one where the measures just vanish, \
 TODO : ensure I use the word 'measure' instead of 'distribution' everwhere
4. Number of steps in Sinkhorn's algorithm. For a reasonable $\epsilon$ we can get it down to ~500, usually not much lower.\
TODO : same as in point 2

Some additional parameters may be set up in the program. Consult LINK TO TRACKER_NOTEBOOK here to to read about those

# Example application
We'll look at data that this program was initially created for. ImageJ was used to locate the points seen in these images.

In [14]:
from skimage import io
from matplotlib import pyplot as plt
from ipywidgets import *
from extract_pos import *


df= import_pos(0,t_max,path='../data/beginning_pos_coarse.csv',px_size=80)

frames=[df.loc[df.t==i].to_dict('records') for i in df.t.unique()]

positions=[np.array([[obs['x'],obs['y']]for obs in observations])for t,observations in enumerate(frames)]

images=io.imread('../data/r2_SuH_exp_50_ext_20_1-Crop1-2000.tif')
t_max=40

x_data=[[]for i in range(t_max+1)]
y_data=[[]for i in range(t_max+1)]
c=[[]for i in range(t_max+1)]

        
for t, ps in enumerate(positions):
    for i,pt in enumerate(ps):
            c[t].append('red')
            x_data[t].append(pt[0])
            y_data[t].append(pt[1])         

def plot_solution(t=0):
    fig,ax=plt.subplots(figsize=(9,12))
    ax.imshow(images[t],cmap='Greys_r')
    ax.scatter([x for x in x_data[t]],[y for y in y_data[t]],c=c[t],s=5)
    ax.set_title(t)

w = interactive(plot_solution, t=(0,t_max))
display(w)

interactive(children=(IntSlider(value=0, description='t', max=40), Output()), _dom_classes=('widget-interact',…

Now let's use the program to track these points

In [18]:
%run tracker.py

Name of the csv with data (with the ".csv"extension). Note it must contain columns named t,x,y,intensity: ../data/beginning_pos_coarse.csv
Maximum frame separation in a connection (Press enter to use default value 1): 2
Weight of entropic regularization term: 0.2
Weight of measure divergence terms: 20
Do you want to use the (slower but more stable) log domain Sinkhorn? (y/n, or enter for default yes): 
Number of iterations for every execution of Sinkhorn's algorithm (Press enter to use default value 500): 
Minimum non-neglebible weight of a conection (Press enter to use default value 1e-5): 
Name of the output file, containing the ".csv" extension (By default `filename`+"_tracked"): 


Computing transport matrices:   1%|1         | 2/149 [00:00<00:17,  8.45it/s]

Creating output csv: 100%|##########| 97/97 [00:00<00:00, 434.82it/s]


The tracks are stored as an additional column 'track_id' in the input 'csv' file. The rows are also sorted by that id and tracks of length 1 are removed\
Let's see them

In [65]:
result=pd.read_csv('../data/beginning_pos_coarse_tracked.csv')
#the new column
t_max=40
result=result.loc[result.t<=t_max]
n_tr=result.track_id.nunique()

#technical bit below. We just need to reassign id's because we skipped some track_id's and we want the numbers to range from 0 to n_tr-1

ids=[int(x) for x in result.track_id.unique()]
ids.sort()

for idx,row in result.iterrows():
    result.loc[idx,'track_id']=ids.index(row.track_id)

In [68]:
import matplotlib.cm as cm

colors = cm.prism(np.linspace(0, 1, n_tr))
x_data=[[]for i in range(t_max+1)]
y_data=[[]for i in range(t_max+1)]
c=[[]for i in range(t_max+1)]
images=io.imread('../data/r2_SuH_exp_50_ext_20_1-Crop1-2000.tif')
scale=80

for index, row in result.iterrows():
    x_data[int(row.t)].append(row.x)
    y_data[int(row.t)].append(row.y)
    c[int(row.t)].append(colors[int(row.track_id)])
    
       
def plot_solution(t=0):
    fig,ax=plt.subplots(figsize=(9,12))
    ax.imshow(images[t],cmap='Greys_r')
    ax.scatter([x/scale for x in x_data[t]],[y/scale for y in y_data[t]],c=c[t],s=5)
    ax.set_title(t)

w = interactive(plot_solution, t=(0,t_max))
display(w)

interactive(children=(IntSlider(value=0, description='t', max=40), Output()), _dom_classes=('widget-interact',…

This seems to work very poorly. Need to play with params a bit. It used to be far better